# Django框架之路由
- 路由urls：作用是根据用户的请求（主要包含url和请求方法）调用相应的处理模块。这个模块就是纯粹的python代码，包含url模式（简单的正则表达式）到python函数（你的视图函数）的简单的映射。此模块就是urls.py文件

#### url映射
- 为什么要去urls.py中寻找视图函数映射
    - 因为在settings.py文件中配置了ROOT_URLCONF为url.py。所有的Django会去urls.py中寻找
- 在urls.py中所有的映射关系，都应该放在urlpatterns这个变量中，否则无法找到


#### Django框架中的两种定义方式
- django.urls.path()
- django.urls.re_path()
- 区别：
    - path：采用字符串匹配模式
    - re_path:可以是字符串或者字符串和正则的组合
    

## 路由详解
#### 路由的访问方式
- 从上往下的逐个匹配，如果匹配到相应的路由，则调用相应的处理模块，匹配终止
- 如果最终没有匹配到，报错
- 如果有子路由，则匹配到相应的父级路由之后，进入子路由重复上面匹配过程
#### 子路由
- Django项目同名文件夹下的urls称为总路由，让负责整个项目的路由分配
- 一般使用的时候推荐的方式是：
    - 项目同名的文件夹下只负责配置分发业务，具体业务根据不同的业务设置不同的APP，即在APP中设置不同的urls.py文件，这种文件就是子路由
- 需要注意的地方：
    - 1.应该使用include函数包含App中的urls.py文件，并且这个urls.py的路径是相对于项目的路径：比如path("book", include("APP名字.urls")）,Include函数需要导入
    - 2.在APP的urls.py文件中，所有url匹配也要放到一个叫做urlpatterns函数中
    - 3.url会根据总路由和子路由的路径进行拼接，所以不要多加斜杠
    
#### path函数
- path标准语法：  path(<route>,<view>,[name=None,**kwargs])
    
           route:表示路径，需要匹配的路径,从端口号后开始到最后一个 “/”为止
           view：表示需要调用的视图函数，必须是一个函数；如果是类的话，要用as_view()转换为函数
           name：别名，为方便其他地方调用和避免重复写长串地址，为view起一个别名
           *kwargs：字典，包含所传递的额外的参数，传到视图函数中时，会作为一个关键字参数传过去
    
#### 关键字参数传递
- 路由可以带参数传递，把参数传递给视图函数
- 比如访问子url时，子路由传递了参数年份，写法：
    - path("sub_tuling/<int:year>"),再在视图函数中把year参数写进去

#### 传递默认参数
- path("sub_tuling/<int:year>", function, {"name":"图灵学院"）,name就是默认参数
- 或者在定义views中的function时就传递参数，def function(...,name="图灵学院")

#### 查询传递的参数
- 采用查询字符串的方式，在url中，不需要单独的匹配查询字符串的部分(但是访问时的url中要在末尾加上  ?参数名=value值)，只需要在视图函数中使用request.GET.get("参数名")的方式即可获取
    

#### path转换器
- path路由使用过程中，会把传入的内容自动转换成了对应类型的参数，比如<int:year>,会把传入的参数自动转换成了整数表示的year参数，其中我们把int叫做path转换器（path Converter）。共有5中类型
    - str：匹配除了路径分隔符(/)之外的非空字符串，默认形式
    - int：匹配0或正整数
    - slug：匹配字母、数字以及横杠、下划线组成的字符串
    - uuid：匹配uuid格式的字符串
    - path：匹配任何非空字符串，包含了路径分隔符
    - 如果该五类转换器无法满足需求，可以自己定义转换器

#### 自定义转换器
- 转换器是一个特殊的类，需要满足三点要求
    - regex：类属性，是字符串类型，表示匹配内容时的正则表达式
    - to_python(self, value)方法:value是由类属性regex所匹配到的字符串，返回具体的python变量值，以提供Django传递到相应的视图函数中
    - to_url(self, value)方法：和to_python相反，value是一个具体的变量值，返回其字符串，通常用于url返向引用
- 可以理解成一个转换器就是有上述三个属性或者函数的类
- 编写自己定义的转换器需要以下几步：
    - 1.新建Converters.py文件(在那个同名文件夹下或APP文件夹下新建)，在文件中编写转换器类（类名一般为xxxConventer）
        - 需导入模块：from django.urls import register_conventer, converters
        - 类编写完成以后， 继续在文件中注册该类：register_converter(类名, 变量名)，变量名用来称呼这个类
    - 2.在app文件下的__init__文件中导入converter.py文件（这样就可以保证类的变量名可以在urls.py中使用）
    - 3.在urls.py文件中使用
        - path("course/<变量名:参数名>/", views.list_course)


#### re_path捕获url中的内容作为参数
- re_path()中可以使用(?P<参数名>参数规则)来捕获url中的参数
    - re_path(r"year/(?P<year>\d{4})/$", views.get_year),

### url反向解析
- url反向解析的本质是对每个url进行命名,然后在编码中使用url命名值
- 具体步骤是；
    - 1.在urls.py(无论是总路由还是子路由，就看你想不想给url命名)文件中的path函数中使用name关键字参数给url命名
            path("tuling/", function, name="url变量名")
    - 2.url命名后如何反转使用
        - a.在HTML页面中
            - 在需要使用url的HTML页面汇总可以使用url标签进行动态的反向解析
            a href="{% url'url名字' 3%"}"> python3编码规范，反向解 </a>
        - b.在python代码中
            - 在views.py文件中的视图函数使用url = reverse("url变量名")，然后使用return redirect(url)即可得到该网页的内容。reverse函数和redirect都需要导入:from django.shortcuts import redirect, reverse
            - 当url需要传递参数的时候，reverse也可以通过kwargs传递键值对参数.reverse("url变量名", kwargs={"name":"taotao"})
            - 如果想要添加字符串的参数，则必须手动进行拼接
                - login_url = reverse("url变量名") + "?next=/"

In [2]:
# 自定义函数举例
## converts文件：
from django.urls import converters, register_converter
class CategoryConverter():
    regex = r"\w+|(\w+\+\w+)+"
    def to_python(self, value):
        # value表示传递进来的url字符串
        result = value.split("+")
        return result
    def to_url(self, value):
        # 这个函数是使用reverse时将python代码转换为url字符串，与to_python恰好相反
        if isinstance(value, list):
            result = "+".join(value)
            return result
        else:
            raise RuntimeError("转换url的参数必须为列表类型")
register_converter(CategoryConverter, "cate")
# converters文件写好后，在APP文件夹下的__init__文件中导入converts文件
# 这样就可以在urls.py文件中直接使用"cate"这个名字
from . import converts

#urls.py文件中
path("course/<cate:categories>/", views.list_course)
# re_path("course/(?P<categories>\w+|(\w+\+\w+)+)/", views.list_course),效果和上面一样


ModuleNotFoundError: No module named 'django'

## django之视图
- 视图概述：
    - 视图即视图模块，接收web请求并返回web响应的事务处理模块
    - 响应指符合HTTP协议要求的任何内容，包括json，string，HTML等
    - 是Django后台的核心内容，一般在每个APP下的views.py文件里面，如果特别复杂可以用别的python文件
    - 视图可以看作是一个接受HTTPResponse的实例，返回一个Response的类的实例或者函数
    - 视图的第一个参数永远都是request，是一个HTTPRequest对象，这个对象存储了请求过来的所有信息。包括携带的参数以及一些头部信息等。在视图中，一般是完成逻辑相关的操作。
    - 视图函数的返回结果必须是HTTPResponse对象或者子类的对象
    


# Debug模式
- 1.如果开启了debug模式，那么以后我们修改了Django项目的代码，然后按下ctrl+s，那么Django就会自动的给我们重启项目，不需要手动重启。
- 2.如果开启了Debug模式，那么以后Django项目中的代码出现了bug，那么在浏览器中就会打印出出错的具体信息
- 3.在上线后，禁止开启debug模式，否则有很大的安全隐患
- 4.如果将debug设置为false，那么必须要设置ALOOWD_HOSTS
    - ALLOWD_HOSTS：这个变量是用来设置以后别人只能通过这个变量汇总的IP地址或域名来进行访问

# 命名空间
## 应用命名空间
- 在多个APP之间，有可能产生同名url，这时候为了避免反转url的时候产生混淆，可以用应用命名空间来做区分。
- 定义步骤：
    - 在APP的urls.py中定义一个叫做app_name的变量，来指定这个应用的命名空间即可
    - 以后在做反转的时候，使用"应用命名空间:url名称"的方式进行反转.比如reverse("App名字:url名字")
    
## 实例命名空间
- 一个APP可以被多个url映射，所以如果只用应用命名空间就会导致在做反转的时候就会产生混淆。所以需要使用实例命名空间。
- 注意：使用实例命名空间必须先指定应用命名空间
- 如何使用：
    - 创建实例命名空间：在include函数中传递一个"namespace"变量即可，该namespace即为实例名字
            path("cms1/, include('cms.urls', namespace='cms1'))
            path("cms2/, include('cms.urls', namespace='cms2'))
    - 获取实例命名空间来指定具体的url
            current_namespace = request.resolver_match.namespace
            return redirect(reverse(current_namespace))

# include函数的用法
- 1.include(module, namespace=None):
    - module:子路由的模块字符串
    - namespace：实例命名空间，设置它必须要先指定应用命名空间
- 2.include((pattern_list, app_namespace), namespace=None)
    - 也可以在元组中设置应用命名空间
- 3.include(pattern_list):pattern_list表示可以将path函数以列表的形式传递进去